In [1]:
import cvxpy as cvx

In [2]:
import numpy as np
import pandas as pd
from numpy.testing import assert_almost_equal

# 均匀分布权重问题

In [4]:
stocks = ['000001', '000002', '000003', '000004', '000005']
alphas = pd.Series([0.1, 0.2, 0., -0.1, -0.2], index=stocks)

In [5]:
w = cvx.Variable(5)

In [6]:
obj = cvx.Maximize(alphas.values.T * w)

In [26]:
cons = [w <= 0.5, w >= -1, cvx.sum(cvx.abs(w)) <= 1]

In [27]:
prob = cvx.Problem(obj, cons)

In [28]:
prob.solve()

0.19999999996286677

In [29]:
np.round(w.value, 4)

array([-0.    ,  0.3641, -0.    ,  0.    , -0.6359])